## Instalação

Este tutorial requer os pacotes `langchain-community` e `pypdf`:


In [1]:
pip install langchain-community pypdf

   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ---------------------------------------- 2.5/2.5 MB 20.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: C:\Users\felip\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


## LangSmith

Muitas das aplicações que você constrói com o LangChain conterão múltiplas etapas com várias chamadas para modelos de linguagem (LLMs). À medida que essas aplicações se tornam mais complexas, torna-se crucial poder inspecionar exatamente o que está acontecendo dentro da sua *chain* ou *agent*. A melhor forma de fazer isso é com o LangSmith.
(necessário se cadastrar no LangSmith)


In [2]:
#export LANGSMITH_TRACING="true"
#export LANGSMITH_API_KEY="..."

Ou, se estiver em um notebook, você pode configurá-las com:


In [3]:
import getpass
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass()

## Documentos e Carregadores de Documentos

O LangChain implementa uma abstração chamada **Document**, que tem como objetivo representar uma unidade de texto e metadados associados. Ela possui três atributos:

- `page_content`: uma string representando o conteúdo;
- `metadata`: um dicionário contendo metadados arbitrários;
- `id`: (opcional) uma string identificadora do documento.

O atributo `metadata` pode capturar informações sobre a origem do documento, sua relação com outros documentos e outras informações. Vale notar que um objeto `Document` individual frequentemente representa um fragmento (*chunk*) de um documento maior.

Podemos gerar documentos de exemplo quando desejado:


In [4]:
from langchain_core.documents import Document

documents = [
    Document(
        page_content="Dogs are great companions, known for their loyalty and friendliness.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"source": "mammal-pets-doc"},
    ),
]

## Carregando Documentos

Vamos carregar um PDF em uma sequência de objetos `Document`. Há um PDF de exemplo no repositório do LangChain. https://github.com/langchain-ai/langchain/tree/master/docs/docs/example_data


In [ ]:
from langchain_community.document_loaders import PyPDFLoader

file_path = "C:/Users/felip/Documents/GitHub/TEEE/tut-06/nke-10k-2023.pdf" # Adjust the path as needed
loader = PyPDFLoader(file_path)

docs = loader.load()

print(len(docs))

107


O `PyPDFLoader` carrega um objeto `Document` por página do PDF. Para cada um deles, podemos acessar facilmente:

- O conteúdo da página como uma string;
- Metadados contendo o nome do arquivo e o número da página.


In [7]:
print(f"{docs[0].page_content[:200]}\n")
print(docs[0].metadata)

Table of Contents
UNITED STATES
SECURITIES AND EXCHANGE COMMISSION
Washington, D.C. 20549
FORM 10-K
(Mark One)
☑  ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(D) OF THE SECURITIES EXCHANGE ACT OF 1934
F

{'producer': 'EDGRpdf Service w/ EO.Pdf 22.0.40.0', 'creator': 'EDGAR Filing HTML Converter', 'creationdate': '2023-07-20T16:22:00-04:00', 'title': '0000320187-23-000039', 'author': 'EDGAR Online, a division of Donnelley Financial Solutions', 'subject': 'Form 10-K filed on 2023-07-20 for the period ending 2023-05-31', 'keywords': '0000320187-23-000039; ; 10-K', 'moddate': '2023-07-20T16:22:08-04:00', 'source': 'C:/Users/felip/Documents/GitHub/TEEE/tut-06/nke-10k-2023.pdf', 'total_pages': 107, 'page': 0, 'page_label': '1'}


## Divisão de Texto (*Splitting*)

Para fins de recuperação de informações e respostas a perguntas, uma página pode ser uma representação muito ampla. Nosso objetivo final será recuperar objetos `Document` que respondam a uma consulta de entrada, e dividir ainda mais nosso PDF ajudará a garantir que os significados das partes relevantes do documento não sejam "diluídos" pelo texto ao redor.

Podemos usar divisores de texto (*text splitters*) para esse propósito. Aqui utilizaremos um divisor de texto simples que particiona com base em caracteres. Dividiremos nossos documentos em blocos de 1000 caracteres com 200 caracteres de sobreposição entre os blocos. A sobreposição ajuda a mitigar a possibilidade de separar uma afirmação de seu contexto importante.

Usamos o `RecursiveCharacterTextSplitter`, que divide recursivamente o documento usando separadores comuns como quebras de linha, até que cada bloco atinja o tamanho apropriado. Esse é o divisor de texto recomendado para casos de uso genéricos com texto.

Configuramos `add_start_index=True` para que o índice de caractere onde cada documento dividido começa dentro do documento original seja preservado como um atributo de metadado chamado `"start_index"`.


In [8]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

len(all_splits)

516

## Embeddings

A busca vetorial (*vector search*) é uma maneira comum de armazenar e pesquisar em dados não estruturados (como texto não estruturado). A ideia é armazenar vetores numéricos associados ao texto. Dada uma consulta, podemos incorporá-la (*embed*) como um vetor da mesma dimensão e usar métricas de similaridade vetorial (como a similaridade cosseno) para identificar textos relacionados.


In [9]:
pip install -qU langchain-google-genai

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: C:\Users\felip\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [10]:
import getpass
import os

if not os.environ.get("GOOGLE_API_KEY"):
  os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")

from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [11]:
vector_1 = embeddings.embed_query(all_splits[0].page_content)
vector_2 = embeddings.embed_query(all_splits[1].page_content)

assert len(vector_1) == len(vector_2)
print(f"Generated vectors of length {len(vector_1)}\n")
print(vector_1[:10])

Generated vectors of length 768

[0.003303560661152005, -0.01885664090514183, -0.023528870195150375, 0.013265608809888363, 0.04694835841655731, 0.04489297419786453, 0.030707117170095444, 0.017642803490161896, 0.0011852466268464923, 0.028473228216171265]


## Armazenamento Vetorial (*Vector Stores*)

Os objetos `VectorStore` do LangChain contêm métodos para adicionar textos e objetos `Document` ao armazenamento, além de permitir consultas utilizando diversas métricas de similaridade. Eles geralmente são inicializados com modelos de *embedding*, que determinam como os dados textuais são convertidos em vetores numéricos.

O LangChain inclui uma variedade de integrações com diferentes tecnologias de armazenamento vetorial. Alguns *vector stores* são hospedados por provedores (por exemplo, vários provedores de nuvem) e requerem credenciais específicas para uso; outros (como o Postgres) funcionam em uma infraestrutura separada que pode ser executada localmente ou via terceiros; e há também aqueles que podem ser executados em memória para cargas de trabalho mais leves.


In [12]:
pip install -qU langchain-core

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: C:\Users\felip\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [13]:
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)

In [14]:
ids = vector_store.add_documents(documents=all_splits)

Observe que a maioria das implementações de armazenamento vetorial permite que você se conecte a um armazenamento vetorial existente — por exemplo, fornecendo um cliente, nome do índice ou outras informações. Consulte a documentação da integração específica para mais detalhes.

Depois de instanciarmos um `VectorStore` que contenha documentos, podemos consultá-lo. O `VectorStore` inclui métodos para consulta:

- De forma síncrona e assíncrona;
- Por consulta em formato de string e por vetor;
- Com ou sem retorno das pontuações de similaridade;
- Por similaridade e relevância marginal máxima (para equilibrar similaridade com a consulta e diversidade nos resultados retornados).

Os métodos geralmente retornarão uma lista de objetos `Document` em seus resultados.


## Uso

Embeddings normalmente representam texto como um vetor "denso", de modo que textos com significados semelhantes ficam geometricamente próximos. Isso nos permite recuperar informações relevantes apenas passando uma pergunta, sem precisar conhecer termos-chave específicos usados no documento.

Retorne documentos com base na similaridade a uma consulta em formato de string:


In [15]:
results = vector_store.similarity_search(
    "How many distribution centers does Nike have in the US?"
)

print(results[0])

page_content='operations. We also lease an office complex in Shanghai, China, our headquarters for our Greater China geography, occupied by employees focused on implementing our
wholesale, NIKE Direct and merchandising strategies in the region, among other functions.
In the United States, NIKE has eight significant distribution centers. Five are located in or near Memphis, Tennessee, two of which are owned and three of which are
leased. Two other distribution centers, one located in Indianapolis, Indiana and one located in Dayton, Tennessee, are leased and operated by third-party logistics
providers. One distribution center for Converse is located in Ontario, California, which is leased. NIKE has a number of distribution facilities outside the United States,
some of which are leased and operated by third-party logistics providers. The most significant distribution facilities outside the United States are located in Laakdal,' metadata={'producer': 'EDGRpdf Service w/ EO.Pdf 22.0.40.0', 

Async query:

In [16]:
results = await vector_store.asimilarity_search("When was Nike incorporated?")

print(results[0])

page_content='Table of Contents
PART I
ITEM 1. BUSINESS
GENERAL
NIKE, Inc. was incorporated in 1967 under the laws of the State of Oregon. As used in this Annual Report on Form 10-K (this "Annual Report"), the terms "we," "us," "our,"
"NIKE" and the "Company" refer to NIKE, Inc. and its predecessors, subsidiaries and affiliates, collectively, unless the context indicates otherwise.
Our principal business activity is the design, development and worldwide marketing and selling of athletic footwear, apparel, equipment, accessories and services. NIKE is
the largest seller of athletic footwear and apparel in the world. We sell our products through NIKE Direct operations, which are comprised of both NIKE-owned retail stores
and sales through our digital platforms (also referred to as "NIKE Brand Digital"), to retail accounts and to a mix of independent distributors, licensees and sales' metadata={'producer': 'EDGRpdf Service w/ EO.Pdf 22.0.40.0', 'creator': 'EDGAR Filing HTML Converter', 'cr

Return scores:

In [17]:
# Note that providers implement different scores; the score here
# is a distance metric that varies inversely with similarity.

results = vector_store.similarity_search_with_score("What was Nike's revenue in 2023?")
doc, score = results[0]
print(f"Score: {score}\n")
print(doc)

Score: 0.7798893082658928

page_content='Table of Contents
FISCAL 2023 NIKE BRAND REVENUE HIGHLIGHTSThe following tables present NIKE Brand revenues disaggregated by reportable operating segment, distribution channel and major product line:
FISCAL 2023 COMPARED TO FISCAL 2022
• NIKE, Inc. Revenues were $51.2 billion in fiscal 2023, which increased 10% and 16% compared to fiscal 2022 on a reported and currency-neutral basis, respectively.
The increase was due to higher revenues in North America, Europe, Middle East & Africa ("EMEA"), APLA and Greater China, which contributed approximately 7, 6,
2 and 1 percentage points to NIKE, Inc. Revenues, respectively.
• NIKE Brand revenues, which represented over 90% of NIKE, Inc. Revenues, increased 10% and 16% on a reported and currency-neutral basis, respectively. This
increase was primarily due to higher revenues in Men's, the Jordan Brand, Women's and Kids' which grew 17%, 35%,11% and 10%, respectively, on a wholesale
equivalent basis.' metad

Retorne documentos com base na similaridade a uma consulta incorporada (*embedded*):


In [18]:
embedding = embeddings.embed_query("How were Nike's margins impacted in 2023?")

results = vector_store.similarity_search_by_vector(embedding)
print(results[0])

page_content='Table of Contents
GROSS MARGIN
FISCAL 2023 COMPARED TO FISCAL 2022
For fiscal 2023, our consolidated gross profit increased 4% to $22,292 million compared to $21,479 million for fiscal 2022. Gross margin decreased 250 basis points to
43.5% for fiscal 2023 compared to 46.0% for fiscal 2022 due to the following:
*Wholesale equivalent
The decrease in gross margin for fiscal 2023 was primarily due to:
• Higher NIKE Brand product costs, on a wholesale equivalent basis, primarily due to higher input costs and elevated inbound freight and logistics costs as well as
product mix;
• Lower margin in our NIKE Direct business, driven by higher promotional activity to liquidate inventory in the current period compared to lower promotional activity in
the prior period resulting from lower available inventory supply;
• Unfavorable changes in net foreign currency exchange rates, including hedges; and
• Lower off-price margin, on a wholesale equivalent basis.
This was partially offset by:'

## Retrievers

Os objetos `VectorStore` do LangChain não são subclasses de `Runnable`. Já os `Retrievers` do LangChain são `Runnables`, ou seja, implementam um conjunto padrão de métodos (por exemplo, operações síncronas e assíncronas, além de operações em lote). Embora possamos construir *retrievers* a partir de *vector stores*, os *retrievers* também podem se conectar a fontes de dados que não sejam armazenamentos vetoriais (como APIs externas).

Podemos criar uma versão simples disso por conta própria, sem precisar estender a classe `Retriever`. Se escolhermos o método que desejamos usar para recuperar documentos, podemos facilmente criar um *runnable*. Abaixo construiremos um usando o método `similarity_search`:


In [19]:
from typing import List

from langchain_core.documents import Document
from langchain_core.runnables import chain


@chain
def retriever(query: str) -> List[Document]:
    return vector_store.similarity_search(query, k=1)


retriever.batch(
    [
        "How many distribution centers does Nike have in the US?",
        "When was Nike incorporated?",
    ],
)

[[Document(id='c010de64-395d-4be7-9b8a-3eb7d8760bbb', metadata={'producer': 'EDGRpdf Service w/ EO.Pdf 22.0.40.0', 'creator': 'EDGAR Filing HTML Converter', 'creationdate': '2023-07-20T16:22:00-04:00', 'title': '0000320187-23-000039', 'author': 'EDGAR Online, a division of Donnelley Financial Solutions', 'subject': 'Form 10-K filed on 2023-07-20 for the period ending 2023-05-31', 'keywords': '0000320187-23-000039; ; 10-K', 'moddate': '2023-07-20T16:22:08-04:00', 'source': 'C:/Users/felip/Documents/GitHub/TEEE/tut-06/nke-10k-2023.pdf', 'total_pages': 107, 'page': 26, 'page_label': '27', 'start_index': 804}, page_content='operations. We also lease an office complex in Shanghai, China, our headquarters for our Greater China geography, occupied by employees focused on implementing our\nwholesale, NIKE Direct and merchandising strategies in the region, among other functions.\nIn the United States, NIKE has eight significant distribution centers. Five are located in or near Memphis, Tennesse

Failed to multipart ingest runs: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')
Failed to multipart ingest runs: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')
Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')
Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Erro

Os *vector stores* implementam um método `as_retriever` que gera um objeto do tipo `Retriever`, especificamente um `VectorStoreRetriever`. Esses *retrievers* incluem atributos específicos como `search_type` e `search_kwargs` que identificam quais métodos do armazenamento vetorial subjacente serão chamados e como serão parametrizados. Por exemplo, podemos replicar o exemplo acima com o seguinte código:


In [20]:
retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 1},
)

retriever.batch(
    [
        "How many distribution centers does Nike have in the US?",
        "When was Nike incorporated?",
    ],
)

[[Document(id='c010de64-395d-4be7-9b8a-3eb7d8760bbb', metadata={'producer': 'EDGRpdf Service w/ EO.Pdf 22.0.40.0', 'creator': 'EDGAR Filing HTML Converter', 'creationdate': '2023-07-20T16:22:00-04:00', 'title': '0000320187-23-000039', 'author': 'EDGAR Online, a division of Donnelley Financial Solutions', 'subject': 'Form 10-K filed on 2023-07-20 for the period ending 2023-05-31', 'keywords': '0000320187-23-000039; ; 10-K', 'moddate': '2023-07-20T16:22:08-04:00', 'source': 'C:/Users/felip/Documents/GitHub/TEEE/tut-06/nke-10k-2023.pdf', 'total_pages': 107, 'page': 26, 'page_label': '27', 'start_index': 804}, page_content='operations. We also lease an office complex in Shanghai, China, our headquarters for our Greater China geography, occupied by employees focused on implementing our\nwholesale, NIKE Direct and merchandising strategies in the region, among other functions.\nIn the United States, NIKE has eight significant distribution centers. Five are located in or near Memphis, Tennesse

O `VectorStoreRetriever` suporta tipos de busca como "similarity" (padrão), "mmr" (máxima relevância marginal, descrito acima) e "similarity_score_threshold". Podemos usar este último para filtrar os documentos retornados pelo *retriever* com base na pontuação de similaridade.

*Retrievers* podem ser facilmente incorporados em aplicações mais complexas, como aplicações de geração aumentada por recuperação (RAG), que combinam uma pergunta com o contexto recuperado para formar um *prompt* para um modelo de linguagem (LLM).
